# Polars

"Polars is an open-source library for data manipulation, known for being one of the fastest data processing solutions on a single machine."

https://pola.rs/

In [1]:
# Install Polars (if necessary)

# !pip install polars

In [2]:
import polars as pl
import datetime as dt

## Getting started

https://docs.pola.rs/user-guide/getting-started/#reading-writing

In [3]:
df = pl.DataFrame(
    {
        "name": ["Alice Archer", "Ben Brown", "Chloe Cooper", "Daniel Donovan"],
        "birthdate": [
            dt.date(1997, 1, 10),
            dt.date(1985, 2, 15),
            dt.date(1983, 3, 22),
            dt.date(1981, 4, 30),
        ],
        "weight": [57.9, 72.5, 53.6, 83.1],  # (kg)
        "height": [1.56, 1.77, 1.65, 1.75],  # (m)
    }
)

In [4]:
df.schema

Schema([('name', String),
        ('birthdate', Date),
        ('weight', Float64),
        ('height', Float64)])

In [5]:
df

name,birthdate,weight,height
str,date,f64,f64
"""Alice Archer""",1997-01-10,57.9,1.56
"""Ben Brown""",1985-02-15,72.5,1.77
"""Chloe Cooper""",1983-03-22,53.6,1.65
"""Daniel Donovan""",1981-04-30,83.1,1.75


### Context: select

In [6]:
result = df.select(
    pl.col("name"),
    pl.col("birthdate").dt.year().alias("birth_year"),
    (pl.col("weight") / (pl.col("height") ** 2)).alias("bmi"),
)

result

name,birth_year,bmi
str,i32,f64
"""Alice Archer""",1997,23.791913
"""Ben Brown""",1985,23.141498
"""Chloe Cooper""",1983,19.687787
"""Daniel Donovan""",1981,27.134694


### Context: with_columns


In [7]:
result = df.with_columns(
    birth_year=pl.col("birthdate").dt.year(),
    bmi=pl.col("weight") / (pl.col("height") ** 2),
)

result

name,birthdate,weight,height,birth_year,bmi
str,date,f64,f64,i32,f64
"""Alice Archer""",1997-01-10,57.9,1.56,1997,23.791913
"""Ben Brown""",1985-02-15,72.5,1.77,1985,23.141498
"""Chloe Cooper""",1983-03-22,53.6,1.65,1983,19.687787
"""Daniel Donovan""",1981-04-30,83.1,1.75,1981,27.134694


### Context: filter

In [8]:
result = df.filter(pl.col("birthdate").dt.year() < 1990)

result

name,birthdate,weight,height
str,date,f64,f64
"""Ben Brown""",1985-02-15,72.5,1.77
"""Chloe Cooper""",1983-03-22,53.6,1.65
"""Daniel Donovan""",1981-04-30,83.1,1.75


In [9]:
result = df.filter(
    pl.col("birthdate").is_between(dt.date(1982, 12, 31), dt.date(1996, 1, 1)),
    pl.col("height") > 1.7,
)

result

name,birthdate,weight,height
str,date,f64,f64
"""Ben Brown""",1985-02-15,72.5,1.77


### Context: group_by


In [10]:
result = df.group_by(
    (pl.col("birthdate").dt.year() // 10 * 10).alias("decade"),
    maintain_order=True,
).len()

result

decade,len
i32,u32
1990,1
1980,3


In [11]:
# we can use agg to compute aggregations over the resulting groups

result = df.group_by(
    (pl.col("birthdate").dt.year() // 10 * 10).alias("decade"),
    maintain_order=True,
).agg(
    pl.len().alias("sample_size"),
    pl.col("weight").mean().round(2).alias("avg_weight"),
    pl.col("height").max().alias("tallest"),
)

result

decade,sample_size,avg_weight,tallest
i32,u32,f64,f64
1990,1,57.9,1.56
1980,3,69.73,1.77


## Titanic example

https://www.kaggle.com/datasets/markmedhat/titanic?resource=download

### Eager execution:

In [12]:
df = pl.read_csv("data/titanic.csv")

In [13]:
df.schema

Schema([('PassengerId', Int64),
        ('Survived', Int64),
        ('Pclass', Int64),
        ('Name', String),
        ('Sex', String),
        ('Age', Float64),
        ('SibSp', Int64),
        ('Parch', Int64),
        ('Ticket', String),
        ('Fare', Float64),
        ('Cabin', String),
        ('Embarked', String)])

In [14]:
# first 5 rows
df.head()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,0,3,"""Braund, Mr. Owen Harris""","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
2,1,1,"""Cumings, Mrs. John Bradley (Fl…","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
3,1,3,"""Heikkinen, Miss. Laina""","""female""",26.0,0,0,"""STON/O2. 3101282""",7.925,null,"""S"""
4,1,1,"""Futrelle, Mrs. Jacques Heath (…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S"""
5,0,3,"""Allen, Mr. William Henry""","""male""",35.0,0,0,"""373450""",8.05,null,"""S"""


In [15]:
print("Summary statistics:\n")
df.describe()

Summary statistics:



statistic,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
str,f64,f64,f64,str,str,f64,f64,f64,str,f64,str,str
"""count""",891.0,891.0,891.0,"""891""","""891""",714.0,891.0,891.0,"""891""",891.0,"""204""","""889"""
"""null_count""",0.0,0.0,0.0,"""0""","""0""",177.0,0.0,0.0,"""0""",0.0,"""687""","""2"""
"""mean""",446.0,0.383838,2.308642,null,null,29.699118,0.523008,0.381594,null,32.204208,null,null
"""std""",257.353842,0.486592,0.836071,null,null,14.526497,1.102743,0.806057,null,49.693429,null,null
"""min""",1.0,0.0,1.0,"""Abbing, Mr. Anthony""","""female""",0.42,0.0,0.0,"""110152""",0.0,"""A10""","""C"""
"""25%""",224.0,0.0,2.0,null,null,20.0,0.0,0.0,null,7.925,null,null
"""50%""",446.0,0.0,3.0,null,null,28.0,0.0,0.0,null,14.4542,null,null
"""75%""",669.0,1.0,3.0,null,null,38.0,1.0,0.0,null,31.0,null,null
"""max""",891.0,1.0,3.0,"""van Melkebeke, Mr. Philemon""","""male""",80.0,8.0,6.0,"""WE/P 5735""",512.3292,"""T""","""S"""


In [16]:
# count missing values in each column

missing_values = df.select([
    pl.col(col).is_null().sum().alias(col) for col in df.columns
])

print("Missing values:\n")
missing_values

Missing values:



PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,177,0,0,0,0,687,2


In [17]:
missing_values.glimpse()

Rows: 1
Columns: 12
$ PassengerId <u32> 0
$ Survived    <u32> 0
$ Pclass      <u32> 0
$ Name        <u32> 0
$ Sex         <u32> 0
$ Age         <u32> 177
$ SibSp       <u32> 0
$ Parch       <u32> 0
$ Ticket      <u32> 0
$ Fare        <u32> 0
$ Cabin       <u32> 687
$ Embarked    <u32> 2



In [18]:
survival_counts = df.group_by("Survived").len()

print("Survival counts:\n")
survival_counts

Survival counts:



Survived,len
i64,u32
0,549
1,342


In [19]:
avg_age_by_class = df.group_by("Pclass").agg([
    pl.col("Age").mean().alias("Average_Age")
])

print("Average age by class:\n")
avg_age_by_class

Average age by class:



Pclass,Average_Age
i64,f64
2,29.87763
3,25.14062
1,38.233441


In [20]:
# Fill missing age with the median age

median_age = df.select(pl.col("Age").median()).item()

df = df.with_columns(
    pl.col("Age").fill_null(median_age)
)

In [21]:
df.head()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,0,3,"""Braund, Mr. Owen Harris""","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
2,1,1,"""Cumings, Mrs. John Bradley (Fl…","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
3,1,3,"""Heikkinen, Miss. Laina""","""female""",26.0,0,0,"""STON/O2. 3101282""",7.925,null,"""S"""
4,1,1,"""Futrelle, Mrs. Jacques Heath (…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S"""
5,0,3,"""Allen, Mr. William Henry""","""male""",35.0,0,0,"""373450""",8.05,null,"""S"""


### Lazy execution

Polars supports two modes of operation: lazy and eager.

https://docs.pola.rs/user-guide/concepts/lazy-api/

Use `scan_csv` instead of `read_csv` to get lazy execution.

Benefits: Polars can use optimizations.

In the lazy API, the query is only evaluated once it is collected. Deferring the execution to the last minute can have significant performance advantages and is why the lazy API is preferred in most cases.

In [22]:
df = pl.scan_csv("data/titanic.csv")

In [23]:
type(df)

polars.lazyframe.frame.LazyFrame

In [24]:
df

In [25]:
# Define the transformation pipeline

result = (
    df
    .with_columns([
        pl.col("Age").fill_null(pl.median("Age")).alias("Age"),
    ])
    .group_by("Pclass")
    .agg([
        pl.col("Age").mean().alias("Average_Age"),
        pl.col("Survived").mean().alias("Survival_Rate"),
        pl.len().alias("Count")
    ])
    .sort("Pclass")
)

In [26]:
print(result.explain())

SORT BY [col("Pclass")]
  AGGREGATE
    [col("Age").mean().alias("Average_Age"), col("Survived").mean().alias("Survival_Rate"), len().alias("Count")] BY [col("Pclass")]
    FROM
     WITH_COLUMNS:
     [col("Age").fill_null([col("Age").median()]).alias("Age")] 
      Csv SCAN [data/titanic.csv]
      PROJECT 3/12 COLUMNS


In [27]:
# Query is executed when it is collected

df_result = result.collect()

In [28]:
df_result

Pclass,Average_Age,Survival_Rate,Count
i64,f64,f64,u32
1,36.81213,0.62963,216
2,29.76538,0.472826,184
3,25.932627,0.242363,491
